# Chains in LangChain

Chains are easily reusable components linked together.

Chains encode a sequence of calls to components like models, document retrievers, other Chains, etc., and provide a simple interface to this sequence.

The Chain interface makes it easy to create apps that are:

* Stateful: add Memory to any Chain to give it state,

* Observable: pass Callbacks to a Chain to execute additional functionality, like logging, outside the main sequence of component calls,

* Composable: combine Chains with other components, including other Chains.

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
## Load you env variables
import os
from dotenv import load_dotenv,find_dotenv
import openai
import anthropic
from anthropic import Anthropic

_ = load_dotenv(find_dotenv(filename='.env'))

In [2]:
## Import Pandas Library
import pandas as pd

## read the cv file from Data folder
projectData = pd.read_csv('./Data/DIM_PROJET.csv')
projectData.head()

,id_projet,Nom_Projet,Jour_Debut,Mois_Debut,Annee_Debut,Jour_Fin,Mois_Fin,Annee_Fin
0,1,Greenview Residences,15,5,2025,15,5,2025
1,2,Riverfront Estates,1,6,2025,1,6,2025
2,3,Sunset Heights,15,6,2025,15,6,2025
3,4,Maplewood Gardens,1,7,2025,1,7,2025
4,5,Lakeview Terrace,15,7,2025,15,7,2025


In [23]:
## Import necessary Langchain's components to interact with the projectData
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import  ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

openai_llm_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.2,
    max_tokens=1024
)

microsoft_phi3_model = HuggingFaceEndpoint(
    repo_id = "microsoft/Phi-3-mini-4k-instruct",
    temperature=0.5,
    token=HUGGINGFACEHUB_API_TOKEN,
    add_to_git_credential=True
)


WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
WARNING! add_to_git_credential is not default parameter.
                    add_to_git_credential was transferred to model_kwargs.
                    Please make sure that add_to_git_credential is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\mehdi\.cache\huggingface\token
Login successful


## LLMChain

In [4]:
## Construct the LLM Chain
question = "Who won the FIFA World Cup in the year 1994? "

## Chain Of thoughts Prompting
template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt,llm=microsoft_phi3_model)

c:\Users\mehdi\anaconda3\envs\langchain-tuto\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [5]:
llm_chain.run(question)

c:\Users\mehdi\anaconda3\envs\langchain-tuto\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' The FIFA World Cup is an international soccer tournament that takes place every four years. The tournament in 1994 was held in the United States. The winner of the 1994 FIFA World Cup was Brazil. They defeated Italy in the final match with a score of 3-2.'

## SimpleSequentialChain - SequentialChain

Another Types of Chain that are aimed to combine multiple chains where the output of the one chain is the input of the next chain.

* SimpleSequentialChain:Single Input/Output.
* SequentialChain:multiple Inputs/Outpus.

In [40]:
from langchain.chains import  SimpleSequentialChain
from langchain.output_parsers import JsonOutputToolsParser

product_category = "Building Muscles"

prompt_one = ChatPromptTemplate.from_template(
    """
    Try to generate me good specific keyword for a product that belongs to a category delimited by triple backticks.\
    Try to think step by step before ansering the final answer.\
    product_category : ```{product_category}```
"""
)
# prompt_one.messages[0].input_variables
first_chain = LLMChain(llm=openai_llm_model,prompt=prompt_one)
first_chain.invoke({
    "product_category":product_category
})

{'product_category': 'Building Muscles',
 'text': '1. Think about the product category: Building Muscles\n2. Consider what types of products are commonly associated with building muscles (e.g. protein supplements, weightlifting equipment, workout programs)\n3. Brainstorm keywords that are specific to products in the building muscles category\n4. Consider the target audience for these products and what keywords they might use when searching for them\n5. Think about the benefits or features of the product that would appeal to someone looking to build muscles\n6. Combine these elements to generate a specific keyword for a product in the building muscles category\n\nFinal keyword: "Muscle-building protein powder"'}

In [41]:
prompt_two = ChatPromptTemplate.from_template(
    """
    Try to generate me a call to action text for a product based on their keywords delimited by triple backticks.\
    Try to think step by step before answering the final answer.\
    keywords : ```{product_keyword}```
"""
)
# prompt_one.messages[0].input_variables
second_chain = LLMChain(llm=openai_llm_model,prompt=prompt_two)

### Combining the chains to construct one SequentialChain

In [42]:
sequetialChains = SimpleSequentialChain(chains=[first_chain,second_chain],verbose=True)
sequetialChains.run(product_category)



> Entering new SimpleSequentialChain chain...
1. Think about the main purpose or benefit of the product within the "Building Muscles" category.
2. Consider any unique features or ingredients that set the product apart.
3. Brainstorm related terms or synonyms that potential customers might use when searching for this type of product.
4. Combine these elements into a specific keyword that accurately describes the product.

Keyword: "Protein-packed Muscle Builder Supplement"
Are you looking to build muscles and increase your strength? Look no further than our Protein-packed Muscle Builder Supplement! With unique ingredients and powerful features, this supplement is designed to help you achieve your fitness goals. Don't wait any longer, try our Protein-packed Muscle Builder Supplement today and take your workouts to the next level!

> Finished chain.


"Are you looking to build muscles and increase your strength? Look no further than our Protein-packed Muscle Builder Supplement! With unique ingredients and powerful features, this supplement is designed to help you achieve your fitness goals. Don't wait any longer, try our Protein-packed Muscle Builder Supplement today and take your workouts to the next level!"